In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFilter
import json
import csv

Annotating a Sample Image

In [2]:
epidural_csv_file = 'segmentations/Results_Epidural Hemorrhage Detection_2020-11-16_21.31.26.148.csv'
epidural_image_repo = 'epidural/max_contrast_window'
epidural_image_file = 'ID_004c4b319.jpg'

In [3]:
def get_coords(image, image_csv_file, image_file_name):
    labels = get_labels_csv(image_csv_file, image_file_name)
    coord_list = []
    for label in labels:
        coord = (label['x'] * image.size[0], label['y'] * image.size[1])
        coord_list.append(coord)
    return coord_list
def get_labels_csv(image_csv_file, image_file_name):
    with open(image_csv_file) as csv_file:
        data_reader = csv.reader(csv_file, dialect='excel')
        for row in data_reader:
            if row[1] == image_file_name:
                return json.loads(row[7])
    return None

In [4]:
im = Image.open(f'{epidural_image_repo}/{epidural_image_file}')
features = []
for x in range(im.size[0]):
    for y in range(20):
        features.append(im.getpixel((x, y))[0])
        features.append(im.getpixel((x, y))[1])
        features.append(im.getpixel((x, y))[2])
draw = ImageDraw.Draw(im)
draw.line(get_coords(im, epidural_csv_file, epidural_image_file), fill='red')
im.show()

In [5]:
# the line of code below loads the 'all labels' field in the csv file in json
# coords_lists = json.loads(row[4].replace("'[", "[").replace("]'", "],").replace(',]', ']'))

Reading an Image and Its Classification

In [6]:
num_features = 786432

In [7]:
all_labels = pd.read_csv('segmentations/hemorrhage-labels.csv')

In [8]:
all_labels

,Image,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
0,ID_000012eaf,0,0,0,0,0,0
1,ID_000039fa0,0,0,0,0,0,0
2,ID_00005679d,0,0,0,0,0,0
3,ID_00008ce3c,0,0,0,0,0,0
4,ID_0000950d7,0,0,0,0,0,0
...,...,...,...,...,...,...,...
752798,ID_ffff82e46,0,0,0,0,0,0
752799,ID_ffff922b9,1,0,0,1,0,0
752800,ID_ffffb670a,1,0,0,0,1,0
752801,ID_ffffcbff8,0,0,0,0,0,0


In [9]:
labels = all_labels[all_labels['any'] == 1]

In [10]:
labels

,Image,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
14,ID_0002081b6,1,0,1,0,0,0
24,ID_0002a38ad,1,0,0,0,1,1
33,ID_000346ce2,1,0,0,0,0,1
36,ID_00042829c,1,0,0,1,0,0
43,ID_0004a5701,1,0,0,0,0,1
...,...,...,...,...,...,...,...
752755,ID_fffc60817,1,0,1,1,0,0
752769,ID_fffd00949,1,0,0,0,1,0
752783,ID_fffe2edb8,1,0,1,1,0,0
752799,ID_ffff922b9,1,0,0,1,0,0


In [11]:
columns = ['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural']

In [12]:
def get_images(classification, max_size):
    classification_labels = labels[(labels[classification] == 1) & (labels[columns].sum(axis=1) == 1)]
    data = np.zeros((classification_labels.shape[0], num_features))
    for index, value in enumerate(classification_labels['Image'].items()):
        data[index, :] = plt.imread(f'{classification}/max_contrast_window/{value[1]}.jpg').reshape(1,-1)
    return data

In [13]:
def get_multi_class_images():
    classification_labels = labels[(labels[columns].sum(axis=1) > 1)]
    data = np.zeros((classification_labels.shape[0], num_features))
    for index, value in enumerate(classification_labels['Image'].items()):
        data[index, :] = plt.imread(f'multi/max_contrast_window/{value[1]}.jpg').reshape(1,-1)
    return data

In [14]:
epidural = get_images('epidural')

In [ ]:
intraparenchymal = get_images('intraparenchymal')

In [ ]:
intraventricular = get_images('intraventricular')

In [ ]:
subarachnoid = get_images('subarachnoid')

In [ ]:
subdural = get_images('subdural')

In [ ]:
multi = get_multi_class_images()

In [15]:
epidural

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [18., 19., 24., ...,  5.,  4.,  9.],
       [38., 37., 51., ...,  0.,  1.,  4.]])

In [ ]:
multi